In [801]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re

# 1. Loading and preparate Data

In [802]:
dataTraining = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)
dataTraining.shape

(7895, 5)

## 1.1 Create y training

In [803]:
###preparate y
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])
print(y_genres)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]]


## 1.2 Linking Data Testing and Data Training

In [804]:
#print(dataTesting.columns.values)
#print(dataTesting.head())
dataTraining=pd.DataFrame(dataTraining)
dataTraining.drop(['genres','rating'], axis='columns', inplace=True)
print(dataTraining.columns.values)
print(dataTesting.columns.values)

['year' 'title' 'plot']
['year' 'title' 'plot']


In [805]:
#print(pd.DataFrame(dataTraining).head())  ### se observan los indices de data training
#pd.concat([pd.DataFrame(dataTraining),pd.DataFrame(dataTesting)],axis=0) ### se observa que en concatenacion 
# los indices que aparecen de primero son los de data training, e.d., data testing queda abajo de data training

print((pd.DataFrame(dataTraining).shape[0],pd.DataFrame(dataTesting).shape[0]))
print(pd.DataFrame(dataTraining).shape[0]+pd.DataFrame(dataTesting).shape[0]) 
union=pd.concat([pd.DataFrame(dataTraining),pd.DataFrame(dataTesting)],axis=0).copy()
print(union.shape)

############ Para extraer data testing de union:

# union.reset_index(inplace=False).iloc[11278-3:11279,]       ## index 11278 dont exists in union (0:11277)


#union.iloc[7895:11278,]  este es el subconjunto para recuperar datatesting
#dataTesting

(7895, 3383)
11278
(11278, 3)


## 1.3 Linking plot and title from union

In [806]:
#print(union['title'].iloc[0:1,])
#print(union['plot'].iloc[0:1,])
#print((union['title']+" "+union['plot']).iloc[0,])
union['text']=pd.DataFrame(union['title']+" "+union['plot'])  ##by row

## 1.4 Grammar  Characterization from union

In [807]:
columnas= pd.DataFrame(columns=['zNoclasif','zCC','zCD','zDT','zEX','zFW','zIN','zJJ','zJJR','zJJS','zLS','zMD','zNN','zNNS','zNNP','zNNPS','zPDT','zPOS','zPRP','zPRP$ ','zRB','zRBR','zRBS','zRP','zTO','zUH','zVB','zVBD','zVBG','zVBN','zVBP','zVBZ','zWDT','zWP','zWP$','zWRB'])

union = pd.concat(list(union.align(columnas)),ignore_index=False)
union=union.reset_index(drop=True)                                      ##!!!
union = union.iloc[0:int(len(union)/2),:]
union=union.reset_index(drop=True)

In [808]:
union['text']=union['text'].map(lambda x: re.sub(r'\W+', ' ', x))

In [ ]:
for h in range(0,len(union)):
    letras=union['text'].iloc[h:h+1].tolist()
    tokens=nltk.word_tokenize(str(letras))
    t_letre=nltk.pos_tag(tokens)
    tablita = pd.DataFrame(t_letre,columns=['Palabra','Tipo'])
    tabla = pd.pivot_table(tablita, values='Palabra', index=['Tipo'], aggfunc='count')
    tabla = pd.DataFrame(data=tabla)
    resultado = tabla['Palabra'].astype(str).str.split(" ", n = 1, expand = True) 
    prueba = resultado.values.tolist()
    df1 = pd.DataFrame(tabla.iloc[:,1:])
    df1 = df1.reset_index()
    df2 = pd.DataFrame(prueba)
    df3 = pd.DataFrame(data=df1,columns=['Tipo','Cantidad'])
    df3['Cantidad']=df2
    gramar = ['CC','CD','DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS','PDT','POS','PRP','PRP$ ','RB','RBR','RBS','RP','TO','UH','VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WP$','WRB']
    a=6
    if (h/len(union)>=0.25 and h/len(union)<=0.255):
        print(h/len(union))
        
    if (h/len(union)>=0.5 and h/len(union)<=0.55):
        print(h/len(union))
        
    if (h/len(union)>=0.9 and h/len(union)<=0.95):
        print(h/len(union))
        
    #print(h)
    #print(len(dataTraining))
    for i in gramar:
        for j in range(0,len(df3)):
            if df3['Tipo'].loc[j] == i: 
                union.iloc[h:h+1,a:a+1] = int(df3['Cantidad'].loc[j])
        a=a+1 
union.fillna(0, inplace=True)

0.2500443341017911
0.2501330023053733
0.2502216705089555
0.25031033871253766
0.2503990069161199
0.2504876751197021
0.25057634332328427
0.2506650115268665
0.25075367973044865
0.2508423479340309
0.25093101613761304
0.25101968434119526
0.2511083525447774
0.25119702074835965
0.2512856889519418
0.25137435715552403
0.25146302535910625
0.2515516935626884
0.25164036176627064
0.2517290299698528
0.251817698173435
0.2519063663770172
0.2519950345805994
0.2520837027841816
0.2521723709877638
0.25226103919134596
0.2523497073949282
0.2524383755985104
0.25252704380209257
0.2526157120056748
0.25270438020925695
0.25279304841283917
0.25288171661642134
0.25297038482000356
0.2530590530235857
0.25314772122716794
0.2532363894307501
0.25332505763433233
0.25341372583791455
0.2535023940414967
0.25359106224507894
0.2536797304486611
0.2537683986522433
0.2538570668558255
0.2539457350594077
0.25403440326298987
0.2541230714665721
0.25421173967015426
0.2543004078737365
0.2543890760773187
0.25447774428090086
0.25456641

## 1.5 Tokenization text feature from union

### Validating consistency data set union:

In [793]:
#union.fillna(0, inplace=True)
print(union.shape)
union.describe()

(11278, 40)


,year,zCC,zCD,zDT,zEX,zFW,zIN,zJJ,zJJR,zJJS,...,zVB,zVBD,zVBG,zVBN,zVBP,zVBZ,zWDT,zWP,zWP$,zWRB
count,11278.000000,11278.0,11278.0,11278.000000,11278.000000,11278.000000,11278.000000,11278.000000,11278.000000,11278.000000,...,11278.000000,11278.000000,11278.000000,11278.000000,11278.000000,11278.000000,11278.000000,11278.000000,11278.000000,11278.000000
mean,1989.555861,0.0,0.0,4.955489,1.128835,13.093988,0.104096,0.082018,14.961607,11.485636,...,0.000798,5.223089,1.823373,2.851835,2.912041,2.689927,7.352101,0.579447,1.210853,0.061979
std,22.700002,0.0,0.0,3.861664,1.330597,8.964654,0.352821,0.522537,10.842739,7.880409,...,0.031222,4.411352,2.405990,2.804647,2.745909,2.730622,6.004704,0.984689,1.641134,0.264623
min,1893.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1979.000000,0.0,0.0,2.000000,0.000000,7.000000,0.000000,0.000000,8.000000,6.000000,...,0.000000,2.000000,0.000000,1.000000,1.000000,1.000000,3.000000,0.000000,0.000000,0.000000
50%,1997.000000,0.0,0.0,4.000000,1.000000,11.000000,0.000000,0.000000,12.000000,10.000000,...,0.000000,4.000000,1.000000,2.000000,2.000000,2.000000,6.000000,0.000000,1.000000,0.000000
75%,2007.000000,0.0,0.0,7.000000,2.000000,17.000000,0.000000,0.000000,19.000000,15.000000,...,0.000000,7.000000,2.000000,4.000000,4.000000,4.000000,10.000000,1.000000,2.000000,0.000000
max,2015.000000,0.0,0.0,84.000000,16.000000,222.000000,5.000000,27.000000,209.000000,110.000000,...,2.000000,84.000000,33.000000,42.000000,40.000000,50.000000,124.000000,10.000000,15.000000,3.000000



### Tfidf

In [794]:
from sklearn.feature_extraction.text import TfidfTransformer
vect = CountVectorizer(ngram_range=(0,3),max_features=2000)
input_text = vect.fit_transform(union['text'])

#############
tfidf_transformer = TfidfTransformer()
input_text = tfidf_transformer.fit_transform(input_text)

In [795]:
###Verificar si funciona3
print(input_text.shape)
print(union.shape)
union=pd.concat([union,pd.DataFrame(input_text.todense())],axis=1).copy()
print(pd.DataFrame(union).shape)

(11278, 2000)
(11278, 40)
(11278, 2040)


## 1.6 Year feature tratated:

In [796]:
union['year1'] = pd.cut(union.year,
                bins=[1890,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010,2020],
                labels=["1890","1900","1910","1920","1930","1940","1950","1960","1970","1980","1990","2000","2010"])

In [797]:
input_year=pd.get_dummies(union['year1'])
input_year.reset_index(inplace=True,drop=True)

In [798]:
input_year.head()

,1890,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010
0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0


# 2. Consolidation input

In [ ]:
union.drop('year1', axis='columns', inplace=True)

In [800]:
input1=pd.concat([union,input_year],axis=1).copy()
input1.shape

(11278, 13)

# 3. Train_Test_Split

In [ ]:
input1.shape

In [ ]:
#input1.head()
datatr=input1.iloc[0:7895,4:input1.shape[1]]
datatest=input1.iloc[7895:11278,4:input1.shape[1]]

In [ ]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(datatr, y_genres, test_size=0.3, random_state=42)

In [ ]:
X_train=X_train.astype(float)
X_test=X_test.astype(float)
y_train_genres=y_train_genres.astype(float)
y_test_genres=y_test_genres.astype(float)

In [ ]:
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#sc.fit_transform(X_train)
#X_test = sc.transform(X_test)
#datatest=sc.transform(datatest).copy()

# 4. Model

In [ ]:
import scipy.sparse as pp
xx=pp.csr_matrix(X_train)
yy=pp.csr_matrix(y_train_genres)
xt=pp.csr_matrix(X_test)

In [ ]:
import xgboost
from xgboost import XGBClassifier
clf1 = OneVsRestClassifier(XGBClassifier(n_estimators=300,max_depth=10,n_jobs=-1, random_state=42,learning_rate=0.02))
clf1.fit(xx,yy)
#y_prob1 = clf1.predict(xt)
y_prob_m1 = clf1.predict_proba(xt)

In [ ]:
clf2 = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=1500, max_depth=10, random_state=42))
clf2.fit(xx,yy)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf3 = OneVsRestClassifier(GradientBoostingClassifier(n_estimators=500),n_jobs=-1)
clf3.fit(xx,yy)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf4 = OneVsRestClassifier(AdaBoostClassifier(n_estimators=200),n_jobs=-1)
clf4.fit(xx,yy) #####
y_prob_m4 = clf4.predict_proba(xt)

In [ ]:
#from sklearn.svm import SVC # "Support Vector Classifier"
#clf = OneVsRestClassifier(SVC(kernel='rbf',probability=True))
#clf.fit(xx,yy) 
#y_prob_m2 = clf.predict_proba(xt)
#del clf

## 4.1 Ensamble

In [ ]:
y_prob_m1 = clf2.predict_proba(xt)
y_prob_m2 = clf3.predict_proba(xt)
y_prob_m3 = clf2.predict_proba(xt)
y_prob_m4 = clf3.predict_proba(xt)

## 4.1 ROC'S

In [ ]:
y_tot=(y_prob_m1 + y_prob_m2 + y_prob_m3 + y_prob_m4)/4
roc_auc_score(y_test_genres, y_tot, average='macro')

# 5 KAGGLE

## 5.1 Predict the testing dataset

In [ ]:
y_prob_m1 = clf2.predict_proba(xt)
y_prob_m2 = clf3.predict_proba(xt)
y_prob_m3 = clf2.predict_proba(xt)
y_prob_m4 = clf3.predict_proba(xt)
ytot=(y_prob_m1+y_prob_m2+y_prob_m3+y_prob_m4)/4

In [ ]:
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

#y_pred_test_genres = clf.predict_proba(datatest.iloc[,4:input1.shape[1]])

In [ ]:
res = pd.DataFrame(y_tot, index=dataTesting.index, columns=cols)

In [ ]:
res.head()

In [ ]:
res.to_csv('pred_genres_text_RF3.csv', index_label='ID')

# API - Rest Service

In [ ]:
# Librerias usadas
import nltk
from nltk import word_tokenize,sent_tokenize
from flask import Flask
from flask_restful import Api, Resource, reqparse
import pandas as pd
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse as pp
from sklearn.preprocessing import MultiLabelBinarizer
import re
import pandas as pd
import numpy as np
import os
import sys
import gc



# Preparate Data

In [ ]:
app = Flask(__name__)
api = Api(app)

dataTraining = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])


In [ ]:
columnas= pd.DataFrame(columns=['zNoclasif','zCC','zCD','zDT','zEX','zFW','zIN','zJJ','zJJR','zJJS','zLS','zMD','zNN','zNNS','zNNP','zNNPS','zPDT','zPOS','zPRP','zPRP$ ','zRB','zRBR','zRBS','zRP','zTO','zUH','zVB','zVBD','zVBG','zVBN','zVBP','zVBZ','zWDT','zWP','zWP$','zWRB'])
gramar = ['CC','CD','DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS','PDT','POS','PRP','PRP$','RB','RBR','RBS','RP','TO','UH','VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WP$','WRB']
vect = CountVectorizer(max_features=2000,ngram_range=(1,50))


# Predict

In [ ]:
class prediccion(Resource):
        def get(self, titulo, plot,ano):
                #Liberar memoria por cada petición
                gc.collect()
                #Carga de modelos
                clf1 = joblib.load(os.path.dirname(__file__) + 'Modelo1.pkl')
                clf2 = joblib.load(os.path.dirname(__file__) + 'Modelo1.pkl')
                clf3 = joblib.load(os.path.dirname(__file__) + 'Modelo1.pkl')
                clf4 = joblib.load(os.path.dirname(__file__) + 'Modelo4.pkl')
                # Tratamiento de titulo, sinapsis y año
                text = titulo+" "+plot
                union=pd.DataFrame(data=[int(ano)],columns=['year'])
                union['year'] = pd.cut(union.year,bins=[1890,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010,2020],labels=["1890","1900","1910","1920","1930","1940","1950","1960","1970","1980","1990","2000","2010"])
                input_year_t=pd.get_dummies(union['year'])
                input_year_t.reset_index(inplace=True,drop=True)
                input_t = pd.DataFrame([text],columns=['text'])
                input_t['text'] = input_t['text'].map(lambda x: re.sub(r'\W+', ' ', x))
                input_t = pd.concat(list(input_t.align(columnas)),ignore_index=False)
                input_t = input_t.reset_index(drop=True)
                input_t = input_t.iloc[0:int(len(input_t)/2),:]
                input_t = input_t.reset_index(drop=True)
                palabras = input_t['text'].iloc[0:1].tolist()
                tokens=nltk.word_tokenize(str(palabras))
                t_letre=nltk.pos_tag(tokens)
                tb1 = pd.DataFrame(t_letre,columns=['Palabra','Tipo'])
                tb2 = pd.pivot_table(tb1, values='Palabra', index=['Tipo'], aggfunc='count')
                rta = tb2['Palabra'].astype(str).str.split(" ", n = 1, expand = True)
                pb = rta.values.tolist()
                df1 = pd.DataFrame(tb1.iloc[:,1:])
                df2 = pd.DataFrame(pb)
                df3 = pd.DataFrame(df1,columns=['Tipo','Cantidad'])
                df3['Cantidad']=df2
                df3['Cantidad'].fillna(0, inplace=True)
                a=1
                for i in gramar:
                        for j in range(0,len(df3)):
                                if df3['Tipo'].loc[j] == i:
                                        input_t.iloc[0:1,a:a+1] = int(df3['Cantidad'].loc[j])
                        a=a+1
                input_t.fillna(0, inplace=True)
                input_text_t = vect.fit_transform(input_t['text'])
                input_gramar_t = input_t.iloc[:,1:37]
                input_t_T=pd.concat([pd.DataFrame(input_gramar_t),pd.DataFrame(input_text_t.todense()),pd.DataFrame(data=[int(ano)]),input_year_t],axis=1,ignore_index=False)
                input_t_T = input_t_T.iloc[0:1,:]
                xt_r = pp.csr_matrix(input_t_T)
                   
                #Predicción    
                y_pred_test_genres_a = clf1.predict_proba(xt_r)
                y_pred_test_genres_b = clf2.predict_proba(xt_r)
                y_pred_test_genres_c = clf3.predict_proba(xt_r)
                y_pred_test_genres_d = clf4.predict_proba(xt_r)
                y_pred_test_genres_t = (y_pred_test_genres_a+y_pred_test_genres_d+y_pred_test_genres_c+y_pred_test_genres_d)/4
                y_pred_test_genres_t.astype(int)
                y_pred_test_genres_t = pd.DataFrame(y_pred_test_genres_t)
                a=y_pred_test_genres_t.replace({0: False, 1: True})
                a=le.classes_[a.values.tolist()]
                a = str(a)
                return a


api.add_resource(prediccion,"/variables/<string:titulo>/<string:plot>/<string:ano>")

app.run(debug=True, use_reloader=False, host='0.0.0.0', port=6800)

In [ ]:
http://ec2-18-191-195-115.us-east-2.compute.amazonaws.com:6800/variables/Most/most is the story of a single father who takes his eight year - old son to work with him at the railroad drawbridge where he is the bridge tender .  a day before ,  the boy meets a woman boarding a train ,  a drug abuser .  at the bridge ,  the father goes into the engine room ,  and tells his son to stay at the edge of the nearby lake .  a ship comes ,  and the bridge is lifted .  though it is supposed to arrive an hour later ,  the train happens to arrive .  the son sees this ,  and tries to warn his father ,  who is not able to see this .  just as the oncoming train approaches ,  his son falls into the drawbridge gear works while attempting to lower the bridge ,  leaving the father with a horrific choice .  the father then lowers the bridge ,  the gears crushing the boy .  the people in the train are completely oblivious to the fact a boy died trying to save them ,  other than the drug addict woman ,  who happened to look out her train window .  the movie ends ,  with the man wandering a new city ,  and meets the woman ,  no longer a drug addict ,  holding a small baby .  other relevant narratives run in parallel ,  namely one of the female drug - addict ,  and they all meet at the climax of this tumultuous film/2003

http://ec2-18-191-195-115.us-east-2.compute.amazonaws.com:6800/variables/Pursuit to Algiers/holmes and watson are recruited in a serpentine fashion to escort the heir to a european throne back to his native country following his father ' s assassination .  because the prince has been educated in great britain ,  holmes persuades him to masquerade as watson ' s nephew nicolas on a ocean liner bound for algiers .  unfortunately the ship is filled with red herrings as well as real assassins ,  and holmes is challenged to outwit them all and deliver his charge to his destination .  among the suspects are a knife - throwing circus performer ,  two shadowy archaeologists ,  a hulking deaf mute ,  an enigmatic ship ' s steward ,  a chanteuse with a mysterious song case ,  and a gun - toting british dowager/1945

http://ec2-18-191-195-115.us-east-2.compute.amazonaws.com:6800/variables/Aftershock/in santiago ,  the american gringo is spending vacation with his chilean friends ariel and the wealthy and influent pollo .  they meet the russian tourists irina ,  the promiscuous kylie and her protective sister monica and pollo invites them to go with his friends and him to valparaiso .  during the night ,  they go to a nightclub but an earthquake collapses the club and there is a warning of tsunami .  the group tries to find a safe place in the middle of the chaos .  meanwhile a nearby prison also collapses and dangerous criminals escape .  when they see the women on the streets ,  they hunt them down along the nightmarish night /2012